<a href="https://colab.research.google.com/github/raj-kariya/rag-workshop/blob/main/rag_workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

rajkariya_sampled_reviews_path = kagglehub.dataset_download('rajkariya/sampled-reviews')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# we are going to study RAG
input_path = '/kaggle/input/sampled-reviews/sampled_reviews.csv'
data = pd.read_csv(input_path)
data

# Sparse Retrieval

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
documents = data['combined'].tolist()

In [ ]:
vectorizer = TfidfVectorizer()

tfidf_matrix = vectorizer.fit_transform(documents)

print("TF-IDF matrix shape:", tfidf_matrix.shape)

In [ ]:
feature_names = vectorizer.get_feature_names_out()
print("Number of features:", len(feature_names))
print("Some feature names:", feature_names[:10])

In [ ]:
dense_rep = tfidf_matrix.toarray()
print("\nTF-IDF vector for first document:")
print(dense_rep[0])

In [ ]:
!pip install rank_bm25


In [ ]:
import numpy as np
from rank_bm25 import BM25Okapi

documents = data['combined'].tolist()

tokenized_docs = [doc.lower().split() for doc in documents]

bm25 = BM25Okapi(tokenized_docs)


In [ ]:
query = "bm25 retrieval function"

query_tokens = query.lower().split()

scores = bm25.get_scores(query_tokens)

print("Query:", query)
for i, score in enumerate(scores):
    print(f"Doc {i} score: {score:.4f} -> {documents[i]}")

In [ ]:
inverted_index = {}

for term_idx, term in enumerate(feature_names):
    col = tfidf_matrix[:, term_idx]
    doc_ids = col.nonzero()[0]  # equivalent to col.indices
    inverted_index[term] = list(doc_ids)

# --------------------------------------------
# Print Inverted Index
# --------------------------------------------
print("\n=== Inverted Index ===")
# for term, doc_list in inverted_index.items():
#     print(f"{term}: {doc_list}")

# --------------------------------------------
# Example Query
# --------------------------------------------
query_terms = ["food", "dog"]
matched_docs = {}

for term in query_terms:
    if term in inverted_index:
        for doc_id in inverted_index[term]:
            matched_docs[doc_id] = matched_docs.get(doc_id, 0) + 1

# Sort documents by number of matching query terms
ranked_results = sorted(matched_docs.items(), key=lambda x: x[1], reverse=True)

print(f"\nQuery: {query_terms}")
print("Ranked Matching Documents:")
for doc_id, match_count in ranked_results:
    print(f"Doc {doc_id} (matches: {match_count}) -> {documents[doc_id]}")

BM 25 method takes worst came O(m^2) which is very bad if you deploy it in production and you wont able to give to give recommended products to users on time.

Now comes transformers which will decreasee the complexity

## Dense Passage Retriever (DPR)

In [ ]:
!pip install transformers

In [ ]:
!pip install faiss-cpu

In [ ]:
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
from sentence_transformers.util import cos_sim
import faiss
import torch


question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
question_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")

passage_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
passage_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

device = "cuda" if torch.cuda.is_available() else "cpu"
question_encoder = question_encoder.to(device)
passage_encoder = passage_encoder.to(device)

In [ ]:
# # -------------------------------
# # Encode Passages with DPR Context Encoder
# # -------------------------------
# def encode_passages(passages):
#     inputs = passage_tokenizer(passages, padding=True, truncation=True, return_tensors="pt", max_length=512).to(device)
#     with torch.no_grad():
#         embeddings = passage_encoder(**inputs).pooler_output  # shape: (n_docs, hidden_size)
#     return embeddings.cpu().numpy()

# passage_embeddings = encode_passages(documents)

In [ ]:
!pip install tqdm

In [ ]:
from tqdm import tqdm

In [ ]:
def encode_passages_batched(passages, batch_size=8):
    all_embeddings = []

    for i in tqdm(range(0, len(passages), batch_size)):
        batch_passages = passages[i:i + batch_size]
        inputs = passage_tokenizer(batch_passages, padding=True, truncation=True, return_tensors="pt", max_length=512).to(device)

        with torch.no_grad():
            outputs = passage_encoder(**inputs)
            embeddings = outputs.pooler_output  # shape: (batch_size, hidden_size)

        all_embeddings.append(embeddings.cpu())

    return torch.cat(all_embeddings, dim=0).numpy()

passage_embeddings = encode_passages_batched(documents, batch_size=128)

spotify annoy library for search

# COLBERT

In [ ]:
data["doc_id"] = data.index
data = data[["doc_id", "combined"]]

In [ ]:
def save_df_to_colbert_format(df, output_path, id_col="doc_id", text_col="combined"):
    with open(output_path, "w", encoding="utf-8") as f:
        for _, row in df.iterrows():
            doc_id = str(row[id_col])
            text = row[text_col].replace("\n", " ").replace("\t", " ").strip()
            f.write(f"{doc_id}\t{text}\n")

# Save to disk
save_df_to_colbert_format(data, "colbert_corpus.tsv")

In [ ]:
!pip install ragatouille

In [ ]:
from ragatouille import RAGPretrainedModel

# Load ColBERTv2
colbert_model = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

# Define your documents


# Index the documents
colbert_model.index(
    collection=documents,
    index_name="my-colbert-index",
    max_document_length=180  # token limit per doc
)